# Closed loop laser scan with NI-DAQmx

## Setup

In [1]:
import time

from pylabnet.utils.logging.logger import LogClient
from pylabnet.scripts.wavemeter import wlm_monitor
from pylabnet.network.client_server import nidaqmx_card
from pylabnet.utils.sweeper import Sweep1D

Enter host and port parameters here:

In [2]:
logger_host = '192.168.0.106'
logger_port = 8267
wavemeter_host = '140.247.189.125'
wavemeter_port = 1477
nidaqmx_host = '140.247.189.125'
nidaqmx_port = 3604

Instantiate all clients

In [3]:
log = LogClient(
    host=logger_host,
    port=logger_port,
    module_tag='laser_scan_closed_loop'
)
# wm = wlm_monitor.Client(
#     host=wavemeter_host,
#     port=wavemeter_port,
# )
# ni = nidaqmx_card.Client(
#     host=nidaqmx_host,
#     port=nidaqmx_port
# )
exp = Sweep1D(logger=log)

## Experiment

Parameters, setup

In [4]:
f0 = 406.693
f1 = 406.695
pts = 101
exp.set_parameters(min=f0, max=f1, pts=pts)
# counter = ni.create_timed_counter(
#     counter_channel='Dev1/Ctr0',
#     physical_channel='/Dev1/PFI0'
# )

Experiment script

In [10]:
def experiment(frequency):

    # Set frequency
    # wm.update_parameters([{
    #     'channel': 1, 'setpoint': frequency
    # }])

    # Activate counter
    # ni.start_timed_counter(counter)
    # time.sleep(0.1)

    # Get counts/sec
    # return ni.get_count(counter)*10
    time.sleep(0.5)
    return frequency
    

In [11]:
exp.configure_experiment(experiment)

In [12]:
exp.run(plot=True)

FigureWidget({
    'data': [{'mode': 'lines', 'type': 'scatter', 'uid': 'cedfc8e0-ab47-4bfc-b60d-e548d0075531'…

## Analyze data

In [8]:
exp.y_data

array([406.693  , 406.69302, 406.69304, 406.69306, 406.69308, 406.6931 ,
       406.69312, 406.69314, 406.69316, 406.69318, 406.6932 , 406.69322,
       406.69324, 406.69326, 406.69328, 406.6933 , 406.69332, 406.69334,
       406.69336, 406.69338, 406.6934 , 406.69342, 406.69344, 406.69346,
       406.69348, 406.6935 , 406.69352, 406.69354, 406.69356, 406.69358,
       406.6936 , 406.69362, 406.69364, 406.69366, 406.69368, 406.6937 ,
       406.69372, 406.69374, 406.69376, 406.69378, 406.6938 , 406.69382,
       406.69384, 406.69386, 406.69388, 406.6939 , 406.69392, 406.69394,
       406.69396, 406.69398, 406.694  , 406.69402, 406.69404, 406.69406,
       406.69408, 406.6941 , 406.69412, 406.69414, 406.69416, 406.69418,
       406.6942 , 406.69422, 406.69424, 406.69426, 406.69428, 406.6943 ,
       406.69432, 406.69434, 406.69436, 406.69438, 406.6944 , 406.69442,
       406.69444, 406.69446, 406.69448, 406.6945 , 406.69452, 406.69454,
       406.69456, 406.69458, 406.6946 , 406.69462, 

## Close counter

In [5]:
ni.close_timed_counter('counter_0')